In [1]:
import torch.nn as nn
import torch

from abl.reasoning import ReasonerBase, KBBase

from abl.learning import BasicNN, ABLModel
from abl.bridge import SimpleBridge
from abl.evaluation import SymbolMetric
from abl.utils import ABLLogger

from examples.models.nn import LeNet5
from examples.mnist_add.datasets.get_mnist_add import get_mnist_add

In [2]:
# Initialize logger
logger = ABLLogger.get_instance("abl")

### Logic Part

In [3]:
# Initialize knowledge base and abducer
class add_KB(KBBase):
    def __init__(self, pseudo_label_list=list(range(10)), max_err=0, use_cache=True):
        super().__init__(pseudo_label_list, max_err, use_cache)

    def logic_forward(self, nums):
        return sum(nums)

kb = add_KB()

# kb = prolog_KB(pseudo_label_list=list(range(10)), pl_file='datasets/mnist_add/add.pl')
abducer = ReasonerBase(kb, dist_func="confidence")

### Machine Learning Part

In [4]:
# Initialize necessary component for machine learning part
cls = LeNet5(num_classes=len(kb.pseudo_label_list))
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(cls.parameters(), lr=0.001, betas=(0.9, 0.99))

In [5]:
# Initialize BasicNN
# The function of BasicNN is to wrap NN models into the form of an sklearn estimator
base_model = BasicNN(
    cls,
    criterion,
    optimizer,
    device,
    save_interval=1,
    batch_size=32,
    num_epochs=1,
)

### Use ABL model to join two parts

In [6]:
# Initialize ABL model
# The main function of the ABL model is to serialize data and 
# provide a unified interface for different machine learning models
model = ABLModel(base_model)

### Metric

In [7]:
# Add metric
metric = [SymbolMetric(prefix="mnist_add")]

### Dataset

In [8]:
# Get training and testing data
train_data = get_mnist_add(train=True, get_pseudo_label=True)
test_data = get_mnist_add(train=False, get_pseudo_label=True)

### Bridge Machine Learning and Logic Reasoning

In [9]:
bridge = SimpleBridge(model, abducer, metric)

### Train and Test

In [12]:
bridge.train(train_data, loops=5, segment_size=10000)
bridge.test(test_data)

11/15 13:36:00 - abl - WARNING - Transform used in the training phase will be used in prediction.


TypeError: Input must be of type list.